In [ ]:
import pandas as pd
import numpy as np

def calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6):
    # Calculate potential PnL for each hour
    df['potential_pnl'] = (df['power_price'] - (df['gas_price'] * heat_rate)) * mw
    df['run_decision'] = False
    df['pnl'] = 0.0

    n = len(df)

    # Function to find the next profitable block starting from 'start'
    def find_next_profitable_period(start):
        for i in range(start, n):
            if i + min_run_hours > n:  # Not enough hours left to fulfill min run time
                return None, 0
            period_pnl = df['potential_pnl'].iloc[i:i + min_run_hours].sum()
            if period_pnl > 0:  # Found a profitable period
                return i, period_pnl
        return None, 0

    current_time = 0
    while current_time < n:
        start, _ = find_next_profitable_period(current_time)
        if start is None:
            break  # No more profitable periods

        # Maximize profitability within this block considering min down time
        max_pnl = 0
        optimal_end = start + min_run_hours
        for end in range(start + min_run_hours, min(start + min_run_hours + min_down_hours, n) + 1):
            period_pnl = df['potential_pnl'].iloc[start:end].sum()
            if period_pnl > max_pnl:
                max_pnl = period_pnl
                optimal_end = end

        # Mark the optimal period as running
        df['run_decision'].iloc[start:optimal_end] = True
        df['pnl'].iloc[start:optimal_end] = df['potential_pnl'].iloc[start:optimal_end]

        # Apply minimum down time constraint before looking for the next profitable period
        current_time = optimal_end + min_down_hours

    return df

# Sample DataFrame setup
data = {
    'time': pd.date_range(start='2022-01-01', periods=24, freq='H'),
    'power_price': np.random.uniform(50, 150, 24),  # Example power prices
    'gas_price': np.random.uniform(3, 10, 24),      # Example gas prices
}
df = pd.DataFrame(data)
df.set_index('time', inplace=True)

# Run the optimized function
optimized_df = calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6)

# Display the optimized DataFrame
print(optimized_df[['power_price', 'gas_price', 'potential_pnl', 'run_decision', 'pnl']])

